In [1]:
# default_exp utils.web

# utils.web

> Web related utils. Supporting screenshot of web page / get webpage contents, scrape webpage etc.

- toc: True

In [2]:
from selenium import webdriver

In [4]:
def create_iphone_driver(driver_path=None):
    iphone = {
        'agent': '--user-agent=Mozilla/5.0 (iPhone; CPU iPhone OS 10_2 like Mac OS X) AppleWebKit/602.3.12 (KHTML, like Gecko) Version/10.0 Mobile/14C92 Safari/602.1',
        'screen_width': 414,
        'screen_height': 736
    }
    options = webdriver.ChromeOptions()
    options.add_argument(iphone['agent'])
    options.add_argument('--headless')
    # needed: see https://stackoverflow.com/questions/50642308/webdriverexception-unknown-error-devtoolsactiveport-file-doesnt-exist-while-t
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    
    chromedriver = driver_path
    driver = webdriver.Chrome(chrome_options=options, executable_path=chromedriver)

    driver.set_window_size(iphone['screen_width'], iphone['screen_height'])
    return driver

In [3]:
def screen_shoot(url):
    SCROLL_PAUSE_TIME = 1
    LOADING_TIME = 3
    driver = create_iphone_driver()
    driver.get(url)

    sleep(LOADING_TIME)
    last_height = 0
    total_height = driver.execute_script("return document.body.scrollHeight;")
    view_height = driver.execute_script("return window.innerHeight;")
    view_width = driver.execute_script("return window.innerHeight;")
    stitched_image = Image.new("RGBA", (view_width, total_height))

    scroll_height = 0
    while scroll_height < total_height:
        driver.execute_script("window.scrollTo(%d, %d)" % (0, scroll_height))
        img = Image.open(io.BytesIO(driver.get_screenshot_as_png()))

        if scroll_height + view_height >= total_height:  # need cropping
            new_height = total_height - scroll_height
            stitched_image.paste(img.crop((0, view_height - new_height, view_width, view_height)), (0, scroll_height))
        else:
            new_height = view_height
            stitched_image.paste(img, (0, scroll_height))
        scroll_height += view_height
        sleep(2)
    image = stitched_image
    page_width = driver.execute_script('return document.body.scrollWidth')
    page_height = driver.execute_script('return document.body.scrollHeight')
    driver.set_window_size(page_width, page_height)
    image = Image.open(io.BytesIO(driver.get_screenshot_as_png()))
    driver.close()
    return image
